In [1]:
import json
import nltk


import torch
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
with open('aspect_cluster.json', 'r') as jfile:
    aspect_json=json.load(jfile)

In [3]:
with open('bags_and_cases_sentence_aspects.json', 'r') as jfile:
    sentence_json=json.load(jfile)

In [4]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [5]:
#assigning sentence to cluster

sent_clust={}
for prod in sentence_json:
    prod_cats={0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], -1:[]}
    for sent in sentence_json[prod]:
        cluster_aspects=aspect_json[prod]
        sentence_aspects=sent[1]
        categs=[]
        for aspect in sentence_aspects:
            for i in range(8):
                if aspect in cluster_aspects[str(i)]:
                    categs.append(i)
        if categs==[]: 
            categs=[-1] #no cluster category
        for cat in set(categs):
            prod_cats[cat].append(sent[0])
    sent_clust[prod]=prod_cats

                
                
        


In [7]:
with open('sent_clust.json', 'w') as jfile:
    json.dump(sent_clust, jfile, indent=4)

In [7]:
#retrieving

salient_sents={}
for prod in tqdm(sent_clust):
    prod_cats={0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], -1:[]}
    for clust in sent_clust[prod]:
        clust_sents = sent_clust[prod][clust]
        if clust_sents==[]:
            continue
        clust_vecs = model.encode(clust_sents)
        centroid = np.mean(clust_vecs, axis=0)
        #print(clust_vecs.shape)
        #print(centroid)
        similarities = cosine_similarity(clust_vecs, [centroid])
        salient_sent = clust_sents[np.argmax(similarities)]
        prod_cats[clust] = salient_sent
    salient_sents[prod] = prod_cats
        


#sent_clust

100%|██████████| 2041/2041 [24:05<00:00,  1.41it/s]


In [8]:
salient_sents

2TOJXNU': {0: 'padding way around gives suitable amount protection computer like look',
  1: 'lrb purse book bag etc rrb carry using strap throw book bag light padded easy access ipad',
  2: 'stylish looking little laptop case well padded well made feels good hand zippers could',
  3: 'makes comfortable wear',
  4: 'let alone laptop pros good protection netbooks',
  5: 'stylish looking little laptop case well padded well made feels good hand zippers could',
  6: 'inside super soft',
  7: 'making adjustable shoulder strap functional fairly comfortable design front exactly pictured centered antique scroll work fabric wet',
  -1: 'found'},
 'B002TR0LUG': {0: 'case durable inside fabric lining',
  1: 'bag fits thinkpad ts perfectly',
  2: 'ordered got disappointment smaller made cheaply returned size description accurate size laptop',
  3: 'seen retail store order amazon',
  4: 'able stop carrying around paper books',
  5: 'seen retail store order amazon',
  6: 'may disappointed looked off

In [9]:
with open('salient_sents.json','w+') as jfile:
    json.dump(salient_sents,jfile,indent=4)